<a href="https://colab.research.google.com/github/mattagnew/SIT796-Reinforcement-Learning/blob/main/Task_2_2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Task 2.2C**

---

In [20]:
%%capture
# INSTALL REQUIRED SYSTEM DEPENDENCIES
!apt-get install -y xvfb x11-utils  
!apt-get install x11-utils > /dev/null 2>&1
!pip install PyOpenGL==3.1.* \
            PyOpenGL-accelerate==3.1.* \
            gym[box2d]==0.17.* 
!pip install pyglet
!pip install ffmpeg
! pip install pyvirtualdisplay
!pip install Image
!pip install gym-maze-trustycoder83

In [21]:
%%capture
!mkdir ./vid
!rm ./vid/*.*

## Environment

---



In [22]:
import sys
# import pygame
import numpy as np
# import math
# import base64
# import io
# import IPython
import gym
import gym_maze

# from gym.wrappers import Monitor
# from IPython import display
from pyvirtualdisplay import Display
from gym.wrappers.monitoring import video_recorder

d = Display()
d.start()

# Recording filename
video_name = "./vid/Practical_2.mp4"

# Setup the environment for the maze
env = gym.make("maze-sample-10x10-v0")

# Setup the video
vid = None
vid = video_recorder.VideoRecorder(env,video_name)

# env = gym.wrappers.Monitor(env,'./vid',force=True)
current_state = env.reset()

## Run

---

In [23]:
# DICTIONARY TO KEEP STATES/COORDS OF THE Q TABLE
states_dic = {} 
count = 0
for i in range(10):
    for j in range(10):
        states_dic[i, j] = count
        count+=1
        
n_actions = env.action_space.n

# NUMBER OF EPISODES
n_episodes = 1000

# MAX NUMBER OF ITERATIONS PER EPISODE
max_iter_episode = 200

# INITIALISE EPSILON TO 0.1
epsilon_array = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5]
epsilon = 0.2

# INITIALISE DISCOUNT FACTOR TO 0.1
gamma_array = [0.5, 0.8, 0.9, 0.95, 0.99]
gamma = 0.99

# SAVE REWARDS
eps_rewards_array = np.zeros((len(epsilon_array), n_episodes))
gam_rewards_array = np.zeros((len(gamma_array), n_episodes))
opt_rewards_array = np.zeros((2, n_episodes))

# ITERATION AVERAGES
iter_verages = np.zeros((len(epsilon_array), max_iter_episode))

# VIDEO FLAG
video_on = 0

### Optimistic vs Not Optimistic

In [ ]:
# # # # # # # # # # # # # # # RUN MAIN ALGORITHM # # # # # # # # # # # # # # #
opt_index = 0
# OPTIMISTIC OR NOT OPTIMISTIC
for opt in np.arange(2):

    print("Optimistic on: ", opt)

    # INITIALISE Q-TABLE TO 0
    Q_table = np.zeros((len(states_dic),n_actions)) + opt * 5

    # EPISODE REWARDS
    opt_rewards = []

    # ITERATE OVER EPISODES
    for e in range(n_episodes):
        
        # NOT DONE
        done = False
        
        # SUM THE REWARDS THAT THE AGENT GETS FROM THE ENVIRONMENT
        rewards = []
        total_episode_reward = 0
        
        # RESET N
        N = 0

        # CONSTANTS STEP SIZE
        N = 10

        for i in range(max_iter_episode): 

            alpha = 1.0/N

            # ONLY CAPTURE VIDEO WHEN NOT LOOPING
            if (video_on > 0):
              env.unwrapped.render() 
              vid.capture_frame()

            current_coordinate_x = int(current_state[0])
            current_coordinate_y = int(current_state[1])
            current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]

            # EXPLORATION
            if np.random.uniform(0,1) < epsilon:
              action = env.action_space.sample()

            # EXPLOITATION  
            else:
              action = int(np.argmax(Q_table[current_Q_table_coordinates]))

            next_state, reward, done, _ = env.step(action)
            
            # GET COORDINATES TO BE USED IN DICTIONARY
            next_coordinate_x = int(next_state[0])
            next_coordinate_y = int(next_state[1])

            # UPDATE Q-TABLE USING THE Q-LEARNING ITERATION
            next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]
            Q_table[current_Q_table_coordinates, action] = \
                            Q_table[current_Q_table_coordinates, action] + alpha * (reward + gamma * max(Q_table[next_Q_table_coordinates,:]) - gamma * Q_table[current_Q_table_coordinates, action])
                
            total_episode_reward = total_episode_reward + reward

            # UPDATE REWARDS
            rewards.append(reward)

            # IF THE EPISODE IS FINISHED, WE LEAVE THE FOR LOOP
            if done:
                break
            current_state = next_state

        opt_rewards.append(rewards)

        # SHOW THE TOTAL EPISODE REWARD IF NOT LOOPING
        if (video_on > 0):        
          print(eps_index,") Epsilon: ",epsilon,". Total episode reward:", total_episode_reward,".")
        
        # RESET THE ENVIRONMENT FOR NEXT EPISODE
        current_state = env.reset()
        
        #rewards_per_episode.append(total_episode_reward)
        opt_rewards_array[opt_index, e] = total_episode_reward

    opt_index += 1
    # SAVE VIDEO IF NOT LOOPING
    if (video_on > 0):
      print("Video successfuly saved.")
      vid.close()
      vid.enabled = False

### Epsilon

In [ ]:
# # # # # # # # # # # # # # # RUN MAIN ALGORITHM # # # # # # # # # # # # # # #
eps_index = 0
# ITERATE OVER EPSILON
for epsilon in epsilon_array:

    print("Epsilon: ", epsilon)

    # INITIALISE Q-TABLE TO 0
    Q_table = np.zeros((len(states_dic),n_actions))

    # EPISODE REWARDS
    ep_rewards = []

    # ITERATE OVER EPISODES
    for e in range(n_episodes):
        
        # NOT DONE
        done = False
        
        # SUM THE REWARDS THAT THE AGENT GETS FROM THE ENVIRONMENT
        rewards = []
        total_episode_reward = 0
        
        # RESET N
        N = 0

        # CONSTANTS STEP SIZE
        N = 10

        for i in range(max_iter_episode): 

            alpha = 1.0/N

            # ONLY CAPTURE VIDEO WHEN NOT LOOPING
            if (video_on > 0):
              env.unwrapped.render() 
              vid.capture_frame()

            current_coordinate_x = int(current_state[0])
            current_coordinate_y = int(current_state[1])
            current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]

            # EXPLORATION
            if np.random.uniform(0,1) < epsilon:
              action = env.action_space.sample()

            # EXPLOITATION  
            else:
              action = int(np.argmax(Q_table[current_Q_table_coordinates]))

            next_state, reward, done, _ = env.step(action)
            
            # GET COORDINATES TO BE USED IN DICTIONARY
            next_coordinate_x = int(next_state[0])
            next_coordinate_y = int(next_state[1])

            # UPDATE Q-TABLE USING THE Q-LEARNING ITERATION
            next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]
            Q_table[current_Q_table_coordinates, action] = \
                            Q_table[current_Q_table_coordinates, action] + alpha * (reward + gamma * max(Q_table[next_Q_table_coordinates,:]) - gamma * Q_table[current_Q_table_coordinates, action])
                
            total_episode_reward = total_episode_reward + reward

            # UPDATE REWARDS
            rewards.append(reward)

            # IF THE EPISODE IS FINISHED, WE LEAVE THE FOR LOOP
            if done:
                break
            current_state = next_state

        ep_rewards.append(rewards)

        # SHOW THE TOTAL EPISODE REWARD IF NOT LOOPING
        if (video_on > 0):        
          print(eps_index,") Epsilon: ",epsilon,". Total episode reward:", total_episode_reward,".")
        
        # RESET THE ENVIRONMENT FOR NEXT EPISODE
        current_state = env.reset()
        
        #rewards_per_episode.append(total_episode_reward)
        eps_rewards_array[eps_index, e] = total_episode_reward

    eps_index += 1
    # SAVE VIDEO IF NOT LOOPING
    if (video_on > 0):
      print("Video successfuly saved.")
      vid.close()
      vid.enabled = False

### Gamma

In [ ]:
# # # # # # # # # # # # # # # RUN MAIN ALGORITHM # # # # # # # # # # # # # # #
gam_index = 0
# ITERATE OVER GAMMA
for gamma in gamma_array:

    print("Gamma: ", gamma)

    # INITIALISE Q-TABLE TO 0
    Q_table = np.zeros((len(states_dic),n_actions))

    # EPISODE REWARDS
    gam_rewards = []

    # ITERATE OVER EPISODES
    for e in range(n_episodes):
        
        # NOT DONE
        done = False
        
        # SUM THE REWARDS THAT THE AGENT GETS FROM THE ENVIRONMENT
        rewards = []
        total_episode_reward = 0
        
        # RESET N
        N = 0

        # CONSTANTS STEP SIZE
        N = 10

        for i in range(max_iter_episode): 

            alpha = 1.0/N

            # ONLY CAPTURE VIDEO WHEN NOT LOOPING
            if (video_on > 0):
              env.unwrapped.render() 
              vid.capture_frame()

            current_coordinate_x = int(current_state[0])
            current_coordinate_y = int(current_state[1])
            current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]

            # EXPLORATION
            if np.random.uniform(0,1) < epsilon:
              action = env.action_space.sample()

            # EXPLOITATION  
            else:
              action = int(np.argmax(Q_table[current_Q_table_coordinates]))

            next_state, reward, done, _ = env.step(action)
            
            # GET COORDINATES TO BE USED IN DICTIONARY
            next_coordinate_x = int(next_state[0])
            next_coordinate_y = int(next_state[1])

            # UPDATE Q-TABLE USING THE Q-LEARNING ITERATION
            next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]
            Q_table[current_Q_table_coordinates, action] = \
                            Q_table[current_Q_table_coordinates, action] + alpha * (reward + gamma * max(Q_table[next_Q_table_coordinates,:]) - gamma * Q_table[current_Q_table_coordinates, action])
                
            total_episode_reward = total_episode_reward + reward

            # UPDATE REWARDS
            rewards.append(reward)

            # IF THE EPISODE IS FINISHED, WE LEAVE THE FOR LOOP
            if done:
                break
            current_state = next_state

        ep_rewards.append(rewards)

        # SHOW THE TOTAL EPISODE REWARD IF NOT LOOPING
        if (video_on > 0):        
          print(eps_index,") Epsilon: ",epsilon,". Total episode reward:", total_episode_reward,".")
        
        # RESET THE ENVIRONMENT FOR NEXT EPISODE
        current_state = env.reset()
        
        #rewards_per_episode.append(total_episode_reward)
        gam_rewards_array[gam_index, e] = total_episode_reward

    gam_index += 1
    # SAVE VIDEO IF NOT LOOPING
    if (video_on > 0):
      print("Video successfuly saved.")
      vid.close()
      vid.enabled = False

## Plots

---

In [27]:
def moving_average(array_in, size_of_window):
    return np.convolve(array_in, np.ones(size_of_window), 'valid') / size_of_window

In [28]:
from matplotlib import pyplot as plt

colours = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

### Epsilon

In [ ]:
moving_average_size = 11
alpha_value = 0.3
shifted_x = np.arange(int((moving_average_size-1)/2),len(moving_average(eps_rewards_array[0,:],moving_average_size))+int((moving_average_size-1)/2))

plt.figure(figsize=(16,8))

ax2 = plt.gca()

eps_0 = plt.plot(shifted_x,moving_average(eps_rewards_array[0,:],moving_average_size),c=colours[0])
eps_1 = plt.plot(shifted_x,moving_average(eps_rewards_array[1,:],moving_average_size),c=colours[1])
eps_2 = plt.plot(shifted_x,moving_average(eps_rewards_array[2,:],moving_average_size),c=colours[2])
eps_5 = plt.plot(shifted_x,moving_average(eps_rewards_array[3,:],moving_average_size),c=colours[3])
eps_10 = plt.plot(shifted_x,moving_average(eps_rewards_array[4,:],moving_average_size),c=colours[4])
eps_20 = plt.plot(shifted_x,moving_average(eps_rewards_array[5,:],moving_average_size),c=colours[5])
eps_50 = plt.plot(shifted_x,moving_average(eps_rewards_array[6,:],moving_average_size),c=colours[6])

plt.plot(eps_rewards_array[0,:],c=colours[0],alpha=alpha_value)
plt.plot(eps_rewards_array[1,:],c=colours[1],alpha=alpha_value)
plt.plot(eps_rewards_array[2,:],c=colours[2],alpha=alpha_value)
plt.plot(eps_rewards_array[3,:],c=colours[3],alpha=alpha_value)
plt.plot(eps_rewards_array[4,:],c=colours[4],alpha=alpha_value)
plt.plot(eps_rewards_array[5,:],c=colours[5],alpha=alpha_value)
plt.plot(eps_rewards_array[6,:],c=colours[6],alpha=alpha_value)

axins2 = ax2.inset_axes([0.15, 0.1, 0.7, 0.5])
axins2.plot(shifted_x,moving_average(eps_rewards_array[0,:],moving_average_size),c=colours[0])
axins2.plot(shifted_x,moving_average(eps_rewards_array[1,:],moving_average_size),c=colours[1])
axins2.plot(shifted_x,moving_average(eps_rewards_array[2,:],moving_average_size),c=colours[2])
axins2.plot(shifted_x,moving_average(eps_rewards_array[3,:],moving_average_size),c=colours[3])
axins2.plot(shifted_x,moving_average(eps_rewards_array[4,:],moving_average_size),c=colours[4])
axins2.plot(shifted_x,moving_average(eps_rewards_array[5,:],moving_average_size),c=colours[5])
axins2.plot(shifted_x,moving_average(eps_rewards_array[6,:],moving_average_size),c=colours[6])
axins2.plot(eps_rewards_array[0,:],c=colours[0],alpha=alpha_value/2)
axins2.plot(eps_rewards_array[1,:],c=colours[1],alpha=alpha_value/2)
axins2.plot(eps_rewards_array[2,:],c=colours[2],alpha=alpha_value/2)
axins2.plot(eps_rewards_array[3,:],c=colours[3],alpha=alpha_value/2)
axins2.plot(eps_rewards_array[4,:],c=colours[4],alpha=alpha_value/2)
axins2.plot(eps_rewards_array[5,:],c=colours[5],alpha=alpha_value/2)
axins2.plot(eps_rewards_array[6,:],c=colours[6],alpha=alpha_value/2)
axins2.set_xlim(0, 50)
axins2.set_ylim(0., 1)
ax2.indicate_inset_zoom(axins2)

plt.legend((eps_0, eps_1, eps_2, eps_5, eps_10, eps_20, eps_50), labels=(r'$\epsilon = 0.0$',r'$\epsilon = 0.01$',r'$\epsilon = 0.02$',r'$\epsilon = 0.05$',r'$\epsilon = 0.1$',r'$\epsilon = 0.2$',r'$\epsilon = 0.5$'), loc="best")

plt.xlabel("Episodes")
plt.ylabel("Average reward")
plt.xlim(0, 1000)
plt.ylim(0, 1)

### Gamma

In [ ]:
moving_average_size = 11
alpha_value = 0.3
shifted_x = np.arange(int((moving_average_size-1)/2),len(moving_average(gam_rewards_array[0,:],moving_average_size))+int((moving_average_size-1)/2))

plt.figure(figsize=(16,8))

ax3 = plt.gca()

gam_5 = plt.plot(shifted_x,moving_average(gam_rewards_array[0,:],moving_average_size),c=colours[0])
gam_8 = plt.plot(shifted_x,moving_average(gam_rewards_array[1,:],moving_average_size),c=colours[1])
gam_9 = plt.plot(shifted_x,moving_average(gam_rewards_array[2,:],moving_average_size),c=colours[2])
gam_95 = plt.plot(shifted_x,moving_average(gam_rewards_array[3,:],moving_average_size),c=colours[3])
gam_99 = plt.plot(shifted_x,moving_average(gam_rewards_array[4,:],moving_average_size),c=colours[4])

plt.plot(gam_rewards_array[0,:],c=colours[0],alpha=alpha_value)
plt.plot(gam_rewards_array[1,:],c=colours[1],alpha=alpha_value)
plt.plot(gam_rewards_array[2,:],c=colours[2],alpha=alpha_value)
plt.plot(gam_rewards_array[3,:],c=colours[3],alpha=alpha_value)
plt.plot(gam_rewards_array[4,:],c=colours[4],alpha=alpha_value)

axins3 = ax3.inset_axes([0.15, 0.1, 0.7, 0.5])
axins3.plot(shifted_x,moving_average(gam_rewards_array[0,:],moving_average_size),c=colours[0])
axins3.plot(shifted_x,moving_average(gam_rewards_array[1,:],moving_average_size),c=colours[1])
axins3.plot(shifted_x,moving_average(gam_rewards_array[2,:],moving_average_size),c=colours[2])
axins3.plot(shifted_x,moving_average(gam_rewards_array[3,:],moving_average_size),c=colours[3])
axins3.plot(shifted_x,moving_average(gam_rewards_array[4,:],moving_average_size),c=colours[4])
axins3.plot(gam_rewards_array[0,:],c=colours[0],alpha=alpha_value/2)
axins3.plot(gam_rewards_array[1,:],c=colours[1],alpha=alpha_value/2)
axins3.plot(gam_rewards_array[2,:],c=colours[2],alpha=alpha_value/2)
axins3.plot(gam_rewards_array[3,:],c=colours[3],alpha=alpha_value/2)
axins3.plot(gam_rewards_array[4,:],c=colours[4],alpha=alpha_value/2)
axins3.set_xlim(0, 50)
axins3.set_ylim(0., 1)
ax3.indicate_inset_zoom(axins3)

plt.legend([gam_5, gam_8, gam_9, gam_95, gam_99], labels=[r'$\gamma = 0.5$',r'$\gamma = 0.8$',r'$\gamma = 0.9$',r'$\gamma = 0.95$',r'$\gamma = 0.99$'], loc="best")

plt.xlabel("Episodes")
plt.ylabel("Average reward")
plt.xlim(0, 1000)
plt.ylim(0, 1)

### Optimistic vs Not Optimistic

In [ ]:
moving_average_size = 11
alpha_value = 0.3
shifted_x = np.arange(int((moving_average_size-1)/2),len(moving_average(opt_rewards_array[0,:],moving_average_size))+int((moving_average_size-1)/2))

plt.figure(figsize=(16,8))

ax = plt.gca()

opt_0 = plt.plot(shifted_x,moving_average(opt_rewards_array[0,:],moving_average_size),c=colours[0])
opt_1 = plt.plot(shifted_x,moving_average(opt_rewards_array[1,:],moving_average_size),c=colours[1])

plt.plot(opt_rewards_array[0,:],c=colours[0],alpha=alpha_value)
plt.plot(opt_rewards_array[1,:],c=colours[1],alpha=alpha_value)

axins = ax.inset_axes([0.15, 0.1, 0.7, 0.5])
axins.plot(shifted_x,moving_average(opt_rewards_array[0,:],moving_average_size),c=colours[0])
axins.plot(shifted_x,moving_average(opt_rewards_array[1,:],moving_average_size),c=colours[1])
axins.plot(opt_rewards_array[0,:],c=colours[0],alpha=alpha_value/2)
axins.plot(opt_rewards_array[1,:],c=colours[1],alpha=alpha_value/2)
axins.set_xlim(0, 50)
axins.set_ylim(0., 1)
ax.indicate_inset_zoom(axins)

plt.legend((opt_0, opt_1), labels=('Not Optimistic', 'Optimistic'), loc="best")

plt.xlabel("Episodes")
plt.ylabel("Average reward")
plt.xlim(0, 1000)
plt.ylim(0.0, 1)